In [1]:
!pip install fastai --upgrade -q

     |████████████████████████████████| 204kB 4.0MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 


In [2]:
from fastai.vision.all import *
import ipywidgets as widgets
import requests

In [3]:

#taken from this StackOverflow answer: https://stackoverflow.com/a/39225039
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"
    session = requests.Session()
    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
    save_response_content(response, destination)
def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None
def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [4]:
file_id = '1-CKjdIjPMVqkQIWLZtHYKavGP8tP3cdm'
destination = './Neck_Line_classifier.pkl'
download_file_from_google_drive(file_id, destination)

## Classifier zu Neckline 

Folgende Modelle werden erkannt:
  

* Bateau
* Carmen
* Closed
* Deep-V-Neck
* Hoodie
* Lapel-collar
* Neckholder
* One-shoulder
* Round
* Serafino-collar
* Shirt-collar
* Spaghetti-straps
* Square Cutout
* Strapless
* Turtleneck
* U-Neck
* V-Neck
* Wide Straps





       -      


In [5]:

path = Path()
inference = load_learner(path/'Neck_Line_classifier.pkl', cpu=True)


In [6]:
btn_upload = widgets.FileUpload()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')
out_pl = widgets.Output()
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(224,224))
    pred,pred_idx,probs = inference.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)
widgets.VBox([widgets.Label('Wählen Sie Ihr Kleid!, damit ich die Vorhersagen machen kann'), 
      btn_upload, btn_run, out_pl, lbl_pred])